In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import chardet 
import operator
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.layers import Dropout, SpatialDropout1D

from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from sklearn.model_selection import train_test_split


In [ ]:

with open("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv", 'rb') as f:
    result = chardet.detect(f.read())
print(result)

In [ ]:
f.close()
#can be tried cp1254, latin and iso-8859-1

In [ ]:

train_data = pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_train.csv", encoding='iso-8859-1')
test_data = pd.read_csv("../input/covid-19-nlp-text-classification/Corona_NLP_test.csv", encoding='iso-8859-1')

In [ ]:
train_data.head()

In [ ]:
train_data.tail()

In [ ]:
test_data.head()

In [ ]:
train_data.isna().sum()

In [ ]:
train_data.shape, test_data.shape

In [ ]:
train_data.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1, inplace=True)

In [ ]:
test_data.drop(['UserName', 'ScreenName', 'Location', 'TweetAt'], axis=1, inplace=True)

In [ ]:
train_data.head()

In [ ]:
train_data.Sentiment.value_counts().plot(kind='bar')

In [ ]:
pd.set_option('display.max_colwidth', None)
train_data.OriginalTweet[0:5]

In [ ]:
#extract hashtag 

train_data["hashtag"] = train_data["OriginalTweet"].apply(lambda x: re.findall(r"#(\w+)", x))

In [ ]:
#data pre processing
#extract url used 
train_data["uri"] = train_data["OriginalTweet"].apply(lambda x : re.findall(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', x))

In [ ]:
train_data["handler"] = train_data["OriginalTweet"].apply(lambda x: re.findall(r"@(\w+)", x))

In [ ]:
#Lemmatize Words


def get_pos_tag(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        # As default pos in lemmatization is Noun
        return wordnet.NOUN

lemmatizer = WordNetLemmatizer()

In [ ]:
pos_tag(["going"])

In [ ]:
#clean the data now 


regex = [
    r'<[^>]+>', #HTML tags
    r'@(\w+)', # @-mentions
    r"#(\w+)", # hashtags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
    r'[^0-9a-z #+_\\r\\n\\t]', #BAD SYMBOLS
]

REPLACE_URLS = re.compile(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+')
REPLACE_HASH = re.compile(r'#(\w+)')
REPLACE_AT = re.compile(r'@(\w+)')
REPLACE_HTML_TAGS = re.compile(r'<[^>]+>')
#REPLACE_DIGITS = re.compile(r'\d+')
#REPLACE_BY = re.compile(r"[/(){}\[\]\|,;.:?\-\'\"$]")
REPLACE_BY = re.compile(r"[^a-z0-9\-]")

STOPWORDS = set(stopwords.words('english'))

#tokens_re = re.compile(r'('+'|'.join(regex)+')', re.VERBOSE | re.IGNORECASE)

sentences = [] #for Word2Vec model

def clean_text(text):
    text = text.lower()
    text = REPLACE_HTML_TAGS.sub(' ', text)
    text = REPLACE_URLS.sub('', text)
    text = REPLACE_HASH.sub('', text)
    text = REPLACE_AT.sub('', text)
    #text = REPLACE_DIGITS.sub(' ', text)
    text = REPLACE_BY.sub(' ', text)
    
    
    text = " ".join(lemmatizer.lemmatize(word.strip(), get_pos_tag(pos_tag([word.strip()])[0][1])) for word in text.split() if word not in STOPWORDS and len(word)>3)
    
    #sentences.append(text.split())
    return (text, text.split())

In [ ]:
train_data["Tweet"], train_data["sentences"] = zip(*train_data["OriginalTweet"].apply(clean_text))

In [ ]:
#from itertools import izip
test_data["Tweet"], test_data["sentences"] = zip(*test_data["OriginalTweet"].apply(clean_text))

In [ ]:
#Now lets do some eda on the data 
#how sentiments are related with the hashtag and user handler
#ext_pos = train_data[train_data['Sentiment'] == 'Extremely Positive']
pos = train_data[train_data["Sentiment"] == "Positive"]
neu = train_data[train_data["Sentiment"] == "Neutral"]
neg = train_data[train_data["Sentiment"] == "Negative"]
#ext_neg = train_data[train_data["Sentiment"] == "Extremely Negative"]

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(20, 20), subplot_kw=dict(aspect="equal"))
for i, param in enumerate(["hashtag", "handler"]): #, "uri"]):
    for j, df in enumerate([ pos, neu, neg]): #, ext_neg, ext_pos,]:
        hash_count = {}
        for tag in df[param]:
            for value in tag:
                hash_count[value] = hash_count.get(value, 0) + 1
        data = sorted(hash_count.items(), key=operator.itemgetter(1))[-10:]
        axs[i,j].pie([value[1] for value in data], labels=[value[0] for value in data], autopct='%1.1f%%', shadow=True, startangle=90)
        axs[i,j].axis('equal')
        axs[i,j].set_title("--Top {0} in {1}--".format(param, df["Sentiment"].values[0].upper()), fontsize=22)
plt.show()

In [ ]:
pd.set_option('display.max_colwidth', None)
train_data.Tweet[50:55]

In [ ]:
#word Cloud

#for color, df in {"green":ext_pos, "yellow":pos, "white":neu, "pink":neg, "orange":ext_neg}.items(): 
for color, df in { "yellow":pos, "white":neu, "pink":neg}.items():
    plt.figure(figsize=(18,18))
    wc_pos = WordCloud(width=400, height=250, min_font_size=5, background_color=color, max_words=10000).generate(" ".join(df["Tweet"]))
    plt.title("word cloud for {0}".format(df["Sentiment"].values[0]), fontsize=25)
    plt.imshow(wc_pos, interpolation = 'bilinear')

In [ ]:
#max len of clean data
import numpy as np
max_len = np.max(train_data["Tweet"].apply(lambda x: len(x)))
max_len

In [ ]:

tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_data["Tweet"].values)

vocab_size = len(tokenizer.word_index) + 1
X = tokenizer.texts_to_sequences(train_data["Tweet"].values)
X = pad_sequences(X, maxlen=max_len, padding='post')

In [ ]:
tokenizer.word_index

In [ ]:
#pre process test data with param of train data
X_test = tokenizer.texts_to_sequences(test_data["Tweet"].values)
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

In [ ]:
!ls ../input/

In [ ]:
# load the GloVe vectors in a dictionary:
#ValueError: could not convert string to float: '.' this is why using ingore
embeddings_index = {}
glovefile = open('../input/glove42b300dtxt/glove.42B.300d.txt','r',encoding='utf-8')
for line in tqdm(glovefile):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
glovefile.close()

print('Found %s word vectors.' % len(embeddings_index))

In [ ]:
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 300))
for word, index in tqdm(tokenizer.word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
X.shape

In [ ]:
X_test.shape

In [ ]:
encoding = {'Extremely Negative': 0,
            'Negative': 0,
            'Neutral': 1,
            'Positive':2,
            'Extremely Positive': 2
           }

labels = ['Negative', 'Neutral', 'Positive']
           

train_data["Sentiment"].replace(encoding, inplace=True)

In [ ]:
test_data["Sentiment"].replace(encoding, inplace=True)

In [ ]:
labels = pd.get_dummies(train_data["Sentiment"]).columns

In [ ]:
y = pd.get_dummies(train_data["Sentiment"]).values
y

In [ ]:
y_test = pd.get_dummies(test_data["Sentiment"]).values
y_test.shape

In [ ]:
#train valid split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=12)
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

In [ ]:
vector_features = 300

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, vector_features, input_length=X.shape[1], weights=[embedding_matrix], trainable=False))
model.add(SpatialDropout1D(0.2))

model.add(Bidirectional(LSTM(300, activation='relu', dropout=0.3, recurrent_dropout=0.3), input_shape=(vector_features, vocab_size)))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.8))

model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
model.summary()

In [ ]:
epochs = 100
batch_size = 512

In [ ]:
history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_data=(X_valid, y_valid),callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
#Model Accuracy
accuracy = history.history["accuracy"]
loss = history.history["loss"]

val_accuracy = history.history["val_accuracy"]
val_loss = history.history["val_loss"]

In [ ]:
print("training acuuracy {0}% and training loss {1}%".format(accuracy[-1]*100, loss[-1]*100))
print("validation acuuracy {0}% and validation loss {1}%".format(val_accuracy[-1]*100, val_loss[-1]*100))

In [ ]:
#plot 

plt.plot(accuracy,'g',label='training accuracy')
plt.plot(val_accuracy, 'r', label='validation accuracy')
plt.legend()
plt.show()


plt.plot(loss,'g',label='training loss')
plt.plot(val_loss, 'r', label='validation loss')
plt.legend()
plt.show()

In [ ]:
#y_pred = model.predict_classes(X_test)
#y_pred = np.argmax(model.predict(X_test), axis=-1)
y_pred = model.predict(X_test)

In [ ]:
print(labels[np.argmax(y_test, 1)][100:120])

In [ ]:
print(labels[np.argmax(y_pred, 1)][100:120])

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(np.argmax(y_test, 1), np.argmax(y_pred, 1))
print(cm)

In [ ]:
sns.heatmap(cm, annot=True)

In [ ]:
#Lets go for Word2Vec
import time
import multiprocessing
from gensim.models import Word2Vec

In [ ]:
cores = multiprocessing.cpu_count()
cores

In [ ]:
w2v_model = Word2Vec(min_count=5,
                     window=5,
                     size=300,
                     sample=6e-5, 
                     alpha=0.01, 
                     min_alpha=0.0007, 
                     negative=7,
                     workers=cores-1)

In [ ]:
#pass list of words of data
#it can be nested list 
t = time.time()
w2v_model.build_vocab(train_data["sentences"], progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time.time() - t) / 60, 2)))

In [ ]:
t = time.time()
w2v_model.train(train_data["sentences"], total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time.time() - t) / 60, 2)))

In [ ]:
#memory efficient
w2v_model.init_sims(replace=True)

In [ ]:
w2v_model.wv.most_similar(positive=["walmart"])

In [ ]:
w2v_model.wv.most_similar(positive=["covid"])

In [ ]:
w2v_model.wv.most_similar(positive=["food"])

In [ ]:
word_vectors = w2v_model.wv
print("length of word vector vocab {}".format(len(word_vectors.vocab)))

In [ ]:
"walmart" in word_vectors.vocab.keys()

In [ ]:
def word2idx(sent):
    sent_id = []
    for word in sent:
        if word in w2v_model.wv.vocab.keys():
            sent_id.append(w2v_model.wv.vocab[word].index)
    return sent_id

In [ ]:
def idx2word(sent_id):
    sent = []
    for idx in sent_id:
       sent.append(w2v_model.wv.index2word[idx])
    return sent

In [ ]:
X_train_w2v = train_data["sentences"].apply(word2idx)
X_test_w2v = test_data["sentences"].apply(word2idx)

In [ ]:
X_train_w2v_pad = pad_sequences(X_train_w2v, maxlen=max_len, padding='post')
X_test_w2v_pad = pad_sequences(X_test_w2v, maxlen=max_len, padding='post')

In [ ]:
X_train_w2v_pad.shape, X_test_w2v_pad.shape

In [ ]:
weights_w2v = w2v_model.wv.syn0
#w2v_model.wv.vectors.shape
vocab_size_w2v, vector_features_w2v = weights_w2v.shape
vocab_size_w2v, vector_features_w2v

In [ ]:
#train valid split

X_train_w2v, X_valid_w2v, y_train_w2v, y_valid_w2v = train_test_split(X_train_w2v_pad, y, test_size=0.20, random_state=12)
print(X_train_w2v.shape, y_train_w2v.shape)
print(X_valid_w2v.shape, y_valid_w2v.shape)

In [ ]:

model_w2v = Sequential()
model_w2v.add(Embedding(vocab_size_w2v, vector_features_w2v, input_length=X_train_w2v_pad.shape[1], weights=[weights_w2v], trainable=False))
model_w2v.add(SpatialDropout1D(0.2))

model_w2v.add(Bidirectional(LSTM(300, activation='relu', dropout=0.3, recurrent_dropout=0.3), input_shape=(vector_features_w2v, vocab_size_w2v)))

model_w2v.add(Dense(1024, activation='relu'))
model_w2v.add(Dropout(0.8))

model_w2v.add(Dense(1024, activation='relu'))
model_w2v.add(Dropout(0.8))

model_w2v.add(Dense(3, activation='softmax'))

model_w2v.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
model_w2v.summary()

In [ ]:
epochs = 100
batch_size = 512

In [ ]:
history = model_w2v.fit(X_train_w2v, y_train_w2v, epochs=epochs, batch_size=batch_size,validation_data=(X_valid_w2v, y_valid_w2v),callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

In [ ]:
#Seems like word2vec was not trained good enough
#may be due  to limited vocab